In [190]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Welcome to my first challenge on kaggle!
In this challenge I would like to do some simple exploration of a cereal data then do a t-test to see if there is any statistically difference in the contain of a certain nutrition value. 

In [191]:
#Import all the library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import probplot
import pylab

In [192]:
# Read the data
df = pd.read_csv('../input/80-cereals/cereal.csv')
df.head()

In [193]:
#Find some info using the data
df.info()

In [194]:
# Summarize the data
df.describe(include = 'all')

* The data consist of three categorical variables : name, mfr, type
* We also have 13 numerical variables
* and it doesn't contain any null value.

In [195]:
#Let's plot all the histogram from this dataset
df.hist(bins=16, figsize=(15, 10), layout=(4, 4))

From the shape of the data, I choose to test **calories**, **protein**, **sodium**, and **carbo**.

Check the distribution using qqplot.
If the variable is normally distributed, the points will be spread close to the line

In [196]:
#Calories
probplot(df["calories"], dist="norm", plot=pylab)

In [197]:
#For protein
probplot(df["protein"], dist="norm", plot=pylab)

In [198]:
#For sodium
probplot(df["sodium"], dist="norm", plot=pylab)

In [199]:
#For carbo
probplot(df["carbo"], dist="norm", plot=pylab)

Looking at the probability plot, variable carbo and sodium are most likely suitable for t-test.

In [200]:
#Let's start from sodium

hot_sodium = df.query('type == "H"')['sodium']
cold_sodium = df.query('type == "C"')['sodium']

#Compare using t-test

print(ttest_ind(hot_sodium, cold_sodium, equal_var=False))

* The variables doesn't seem to have equal variance so I use equal_var = False
* p-value = 0.024 less than significant p-value 0.05 so we reject the null hypothesis
* reject the null (hypotheses that the two has no different value)
* hot and cold cereals have significantly difference in the sodium content

In [201]:
print("Mean sodium for hot cereals: ", hot_sodium.mean())
print("Mean sodium for cold cereals ", cold_sodium.mean())

In [202]:
plt.hist(cold_sodium, alpha = 0.5, label = "cold")
plt.hist(hot_sodium, alpha = 0.5, label = "hot")
plt.title("Histrogram of cold and hot cereals sodium contain")
plt.legend()
plt.show()

equal_varbool, optional
If True (default), perform a standard independent 2 sample test that assumes equal population variances [1]. If False, perform Welch’s t-test, which does not assume equal population variance [2].

In [203]:
#Let's start from carbo
hot_carbo = df.query('type == "H"')['carbo']
cold_carbo = df.query('type == "C"')['carbo']

#Compare using t-test

print(ttest_ind(hot_carbo, cold_carbo, equal_var=False))

* p-value = 0.72 a lot more than significant p-value 0.05 so we can'treject the null hypothesis
* hot and cold cereals have not much difference in carbo content

In [204]:
# What about the mean between carbo nutrition of hot cereal and cold cereal
print("Mean carbo for hot cereals: ", hot_carbo.mean())
print("Mean carbo for cold cereals ", cold_carbo.mean())

In [205]:
#Plot the histogram of carbo
plt.hist(cold_carbo, alpha = 0.5, label = "cold")
plt.hist(hot_carbo, alpha = 0.5, label = "hot")
plt.title("Histrogram of cold and hot cereals carbo contain")
plt.legend()
plt.show()

#### Conclusion
There is a significant difference of sodium contain between hot and cold cereals, however they contain pretty much similar carbohydrate content. 

To reduce the variance between hot and cold cereals, we need more nutrition data of hot cereals.

This challenge is part of data challenge for beginner [Click here for more](https://www.kaggle.com/rtatman/the-5-day-data-challenge/notebook). 

Thank you for reading. I will appreciate inputs and comments :D